In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import os

In [2]:
# 1. Daten laden
required_files = ["umsatzdaten_gekuerzt.csv", "kiwo.csv", "wetter.csv", "Data Import and Preparation/feiertage.csv"]
for file in required_files:
    if not os.path.exists(file):
        raise FileNotFoundError(f"Datei '{file}' nicht gefunden.")

umsatzdaten = pd.read_csv("umsatzdaten_gekuerzt.csv")
kiwo_data = pd.read_csv("kiwo.csv")
wetter_data = pd.read_csv("wetter.csv")
feiertage = pd.read_csv("Data Import and Preparation/feiertage.csv")

In [3]:
# 2. Datum anpassen und konvertieren
umsatzdaten['Datum'] = pd.to_datetime(umsatzdaten['Datum'], errors='coerce')
kiwo_data['Datum'] = pd.to_datetime(kiwo_data['Datum'], errors='coerce')
wetter_data['Datum'] = pd.to_datetime(wetter_data['Datum'], errors='coerce')

# Feiertags-Datum anpassen
feiertage['Datum'] = feiertage['Datum'].apply(
    lambda x: f"{str(x).strip()} 2017" if isinstance(x, str) else None
)
feiertage['Datum'] = pd.to_datetime(feiertage['Datum'], format='%d. %b %Y', errors='coerce')

In [4]:
# 3. Daten zusammenführen
merged_data = umsatzdaten.merge(kiwo_data, on='Datum', how='left')
merged_data = merged_data.merge(wetter_data, on='Datum', how='left')
merged_data = merged_data.merge(feiertage[['Datum', 'Name']], on='Datum', how='left')

In [5]:
# 4. Zusätzliche Variablen erstellen
merged_data['Wochentag'] = merged_data['Datum'].dt.day_name()
merged_data['Wochentag_Nummer'] = merged_data['Datum'].dt.weekday
merged_data['Monat'] = merged_data['Datum'].dt.month
merged_data['Quartal'] = merged_data['Datum'].dt.quarter
merged_data['Ist_Wochenende'] = merged_data['Wochentag'].isin(['Saturday', 'Sunday']).astype(int)
merged_data['Ist_Feiertag'] = merged_data['Name'].notnull().astype(int)

def categorize_temperature(temp):
    if temp < 10:
        return "Kalt"
    elif 10 <= temp < 20:
        return "Mild"
    else:
        return "Warm"

merged_data['Temperatur_Kategorie'] = merged_data['Temperatur'].apply(categorize_temperature)
merged_data['Feiertagsnaehe'] = (
    (merged_data['Ist_Feiertag'] == 1) |
    (merged_data['Datum'] - pd.Timedelta(days=1)).isin(feiertage['Datum']) |
    (merged_data['Datum'] + pd.Timedelta(days=1)).isin(feiertage['Datum'])
).astype(int)
merged_data['Umsatz_Trend_7_Tage'] = merged_data['Umsatz'].rolling(window=7, min_periods=1).mean()

# Fehlende Werte behandeln
merged_data.fillna(0, inplace=True)


In [6]:
# 5. Datensatz teilen
train_data = merged_data[(merged_data['Datum'] >= '2013-07-01') & (merged_data['Datum'] <= '2017-07-31')]
validation_data = merged_data[(merged_data['Datum'] >= '2017-08-01') & (merged_data['Datum'] <= '2018-07-31')]

In [7]:
# 6. Lineares Modell erstellen
features = ['Wochentag_Nummer', 'Feiertagsnaehe', 'Temperatur', 'Ist_Wochenende']
X_train = train_data[features]
y_train = train_data['Umsatz']

model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [8]:
# 7. Adjustiertes R² berechnen
n = len(y_train)
p = X_train.shape[1]
r2 = model.score(X_train, y_train)
adj_r2 = 1 - ((1 - r2) * (n - 1) / (n - p - 1))

# Ergebnisse ausgeben
print("R²:", r2)
print("Adjustiertes R²:", adj_r2)

R²: 0.07196908746722364
Adjustiertes R²: 0.07147532268268908


In [9]:
# 8. Ergebnisse speichern
output_dir = "Baseline Model"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Ergebnisse dokumentieren
with open(os.path.join(output_dir, "results.txt"), "w") as f:
    f.write(f"R²: {r2}\n")
    f.write(f"Adjustiertes R²: {adj_r2}\n")

train_data.to_csv(os.path.join(output_dir, "train_data.csv"), index=False)
validation_data.to_csv(os.path.join(output_dir, "validation_data.csv"), index=False)

In [10]:
# 9. Sample Submission laden
submission_file = "sample_submission.csv"
if not os.path.exists(submission_file):
    raise FileNotFoundError(f"Datei '{submission_file}' nicht gefunden.")

submission = pd.read_csv(submission_file)

# IDs mit Validierungsdaten verknüpfen
submission['id'] = range(1, len(submission) + 1)
validation_data['id'] = range(1, len(validation_data) + 1)

# Umsatz mit dem Modell vorhersagen
X_val = validation_data[features]
y_pred = model.predict(X_val)

# Sicherstellen, dass die Länge übereinstimmt
if len(y_pred) != len(submission):
    y_pred = y_pred[:len(submission)]

# Umsatzwerte in die Submission-Datei einfügen
submission['Umsatz'] = y_pred

# Datum-Spalte entfernen
if 'Datum' in submission.columns:
    submission = submission.drop(columns=['Datum'])

# Submission speichern
submission.to_csv("submission.csv", index=False)

print("Sample Submission aktualisiert und gespeichert als 'submission.csv'.")
print("Berechnungen abgeschlossen und Ergebnisse gespeichert.")

Sample Submission aktualisiert und gespeichert als 'submission.csv'.
Berechnungen abgeschlossen und Ergebnisse gespeichert.


C:\Users\ayham\AppData\Local\Temp\ipykernel_36344\1535706858.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  validation_data['id'] = range(1, len(validation_data) + 1)
